<a href="https://colab.research.google.com/github/rimaakhmedov/nlp-itis/blob/main/homework_1/NLP_hw1%262.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Классификация отзывов на положительные/отрицательные используя

## Импорт библиотек

In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
import pandas as pd
import re
from collections import Counter

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Data/labeledTrainData.tsv', sep='\t', encoding='utf-8')
df

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...
...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...
24996,5064_1,0,I don't believe they made this film. Completel...
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...


In [4]:
max_words = 10000

## Токенизация

In [5]:
df_copy1 = df.copy()

In [6]:
def space_tokenize(text):
  tokenized = text.split()
  return tokenized

In [7]:
df_copy1['tokenized_text'] = df_copy1['review'].apply(space_tokenize)
df_copy1

,id,sentiment,review,tokenized_text
0,5814_8,1,With all this stuff going down at the moment w...,"[With, all, this, stuff, going, down, at, the,..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[\The, Classic, War, of, the, Worlds\"", by, Ti..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"[The, film, starts, with, a, manager, (Nichola..."
3,3630_4,0,It must be assumed that those who praised this...,"[It, must, be, assumed, that, those, who, prai..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"[Superbly, trashy, and, wondrously, unpretenti..."
...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,"[It, seems, like, more, consideration, has, go..."
24996,5064_1,0,I don't believe they made this film. Completel...,"[I, don't, believe, they, made, this, film., C..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","[Guy, is, a, loser., Can't, get, girls,, needs..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,"[This, 30, minute, documentary, Buñuel, made, ..."


## Даем каждому токену свой код

In [8]:
words = Counter()

In [9]:
for txt in df_copy1['tokenized_text']:
  words.update(txt)

In [10]:
word_to_index = dict()
index_to_word = dict()

In [11]:
for i, word in enumerate(words.most_common(max_words-2)):
  word_to_index[word[0]] = i + 2
  index_to_word[i + 2] = word[0]

In [12]:
def text_to_sequence(txt, word_to_index):
  seq = []
  for word in txt:
    index = word_to_index.get(word, 1)
    if index != 1:
      seq.append(index)
  return seq

In [13]:
df_copy1['sequences'] = df_copy1.apply(lambda row: text_to_sequence(row['tokenized_text'], word_to_index), axis = 1)

In [14]:
df_copy1

,id,sentiment,review,tokenized_text,sequences
0,5814_8,1,With all this stuff going down at the moment w...,"[With, all, this, stuff, going, down, at, the,...","[555, 37, 11, 746, 168, 215, 31, 2, 710, 16, 3..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[\The, Classic, War, of, the, Worlds\"", by, Ti...","[701, 6489, 1183, 5, 2, 30, 3969, 9724, 7, 3, ..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"[The, film, starts, with, a, manager, (Nichola...","[19, 24, 467, 16, 3, 4145, 679, 3690, 5918, 6,..."
3,3630_4,0,It must be assumed that those who praised this...,"[It, must, be, assumed, that, those, who, prai...","[56, 212, 27, 5485, 10, 151, 35, 7271, 11, 24,..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"[Superbly, trashy, and, wondrously, unpretenti...","[5252, 4, 1735, 19, 608, 1045, 583, 195, 2, 30..."
...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,"[It, seems, like, more, consideration, has, go...","[56, 180, 36, 48, 39, 1002, 74, 2, 1277, 972, ..."
24996,5064_1,0,I don't believe they made this film. Completel...,"[I, don't, believe, they, made, this, film., C...","[9, 96, 262, 38, 97, 11, 150, 19, 90, 24, 14, ..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","[Guy, is, a, loser., Can't, get, girls,, needs...","[3812, 7, 3, 7090, 75, 4089, 707, 6, 1737, 997..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,"[This, 30, minute, documentary, Buñuel, made, ...","[49, 1253, 1012, 832, 97, 8, 2, 384, 7880, 42,..."


## Готовим данные к обучению

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
train, test = train_test_split(df_copy1, test_size=0.2)

In [17]:
X_train_seq = train['sequences']
y_train = train['sentiment']

In [18]:
X_train_seq

13250    [416, 2, 608, 5658, 306, 3, 426, 396, 17, 42, ...
24886    [49, 21, 135, 26, 15, 2046, 723, 12, 7, 131, 4...
3070     [6078, 13, 171, 58, 319, 2, 394, 6, 4, 431, 6,...
21584    [255, 64, 2704, 9193, 9, 200, 2, 902, 329, 14,...
15782    [234, 238, 15, 2, 7892, 457, 5542, 601, 11, 7,...
                               ...                        
6046     [265, 30, 8, 17, 9458, 39, 6, 3, 2816, 6481, 4...
20170    [19, 125, 22, 9961, 4, 3016, 16, 358, 101, 488...
20474    [40, 7755, 141, 239, 11, 832, 6, 27, 772, 4, 5...
22611    [884, 5532, 36, 726, 16, 616, 2087, 2, 3127, 2...
24215    [24, 7, 185, 2172, 2, 54, 179, 6890, 84, 34, 6...
Name: sequences, Length: 20000, dtype: object

In [19]:
y_train

13250    0
24886    0
3070     0
21584    1
15782    1
        ..
6046     0
20170    0
20474    0
22611    0
24215    0
Name: sentiment, Length: 20000, dtype: int64

In [20]:
X_test_seq = test['sequences']
y_test = test['sentiment']

## Создаем BoW

In [21]:
import numpy as np

In [22]:
def vectorize_sequences(sequences, dimension=10000):
  results = np.zeros((len(sequences), dimension))
  for i, sequence in enumerate(sequences):
    for index in sequence:
      results[i, index] += 1.
  return results

In [23]:
X_train = vectorize_sequences(X_train_seq, max_words)
X_test = vectorize_sequences(X_test_seq, max_words)

In [24]:
X_train[0][:100]

array([0., 0., 3., 8., 6., 2., 1., 0., 2., 0., 0., 0., 0., 2., 0., 0., 0.,
       2., 0., 1., 1., 0., 3., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       0., 0., 0., 0., 1., 1., 0., 0., 1., 0., 2., 0., 0., 1., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [25]:
len(X_train[0])

10000

## Обучение модели без предобработки

In [26]:
class NaiveBayesClassifier:
    def __init__(self, n_classes):
        self.n_classes = n_classes
        self.class_probs = {}
        self.conditional_probabilities = {}

    def fit(self, X, y):
        unique_classes, class_counts = np.unique(y, return_counts=True)
        total_samples = len(y)

        for i in range(self.n_classes):
            class_prob = class_counts[i] / total_samples
            self.class_probs[unique_classes[i]] = class_prob

        for class_label in unique_classes:
            class_samples = X[y == class_label]
            total_class_samples = len(class_samples)

            self.conditional_probabilities[class_label] = {}

            for word_index in range(len(class_samples[0])):
                word_count = np.sum(class_samples[:, word_index])
                word_prob = (word_count + 1) / (total_class_samples + 2)
                self.conditional_probabilities[class_label][word_index] = word_prob

    def predict(self, X):
        predictions = []

        for sample in X:
            best_class = None
            max_log_prob = -np.inf

            for class_label in self.class_probs.keys():
                log_prob = np.log(self.class_probs[class_label])

                for word_index in range(len(sample)):
                    if sample[word_index] == 1:
                        log_prob += np.log(self.conditional_probabilities[class_label][word_index])

                if log_prob > max_log_prob:
                    max_log_prob = log_prob
                    best_class = class_label

            predictions.append(best_class)

        return predictions

In [27]:
classifier = NaiveBayesClassifier(2)

classifier.fit(X_train, y_train)

In [28]:
y_pred = classifier.predict(X_test)

## Метрики без предобработки

In [29]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.85      0.85      0.85      2512
           1       0.85      0.85      0.85      2488

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Balanced_accuracy:", balanced_accuracy)

Accuracy: 0.849
Balanced_accuracy: 0.8489960161287664


## Предобработка

In [31]:
df_copy2 = df.copy()

In [32]:
df_copy2.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [33]:
def preprocess(text):
  text = re.sub(r'<.*?>', '', text)
  text = text.lower()
  text = re.findall(r'\b\w+\b|[.,!?;]', text)
  preprocessed_text = [word for word in text if word.isalnum()]
  return preprocessed_text

In [34]:
df_copy2['preprocessed_text'] = df_copy2['review'].apply(preprocess)
df_copy2

,id,sentiment,review,preprocessed_text
0,5814_8,1,With all this stuff going down at the moment w...,"[with, all, this, stuff, going, down, at, the,..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[the, classic, war, of, the, worlds, by, timot..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"[the, film, starts, with, a, manager, nicholas..."
3,3630_4,0,It must be assumed that those who praised this...,"[it, must, be, assumed, that, those, who, prai..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"[superbly, trashy, and, wondrously, unpretenti..."
...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,"[it, seems, like, more, consideration, has, go..."
24996,5064_1,0,I don't believe they made this film. Completel...,"[i, don, t, believe, they, made, this, film, c..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","[guy, is, a, loser, can, t, get, girls, needs,..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,"[this, 30, minute, documentary, buñuel, made, ..."


## Стемминг

In [35]:
from nltk.stem import PorterStemmer

In [36]:
stemmer = PorterStemmer()

In [37]:
def stemming(text):
    stem_words = [stemmer.stem(word) for word in text]
    return stem_words

In [38]:
df_copy2['preprocessed_text'] = df_copy2['preprocessed_text'].apply(stemming)
df_copy2.head()

,id,sentiment,review,preprocessed_text
0,5814_8,1,With all this stuff going down at the moment w...,"[with, all, thi, stuff, go, down, at, the, mom..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[the, classic, war, of, the, world, by, timoth..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"[the, film, start, with, a, manag, nichola, be..."
3,3630_4,0,It must be assumed that those who praised this...,"[it, must, be, assum, that, those, who, prais,..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"[superbl, trashi, and, wondrous, unpretenti, 8..."


## Даем каждому токену свой код 2

In [39]:
words = Counter()

In [40]:
for txt in df_copy2['preprocessed_text']:
  words.update(txt)

In [41]:
word_to_index = dict()
index_to_word = dict()

In [42]:
for i, word in enumerate(words.most_common(max_words-2)):
  word_to_index[word[0]] = i + 2
  index_to_word[i + 2] = word[0]

In [43]:
def text_to_sequence(txt, word_to_index):
  seq = []
  for word in txt:
    index = word_to_index.get(word, 1)
    if index != 1:
      seq.append(index)
  return seq

In [44]:
df_copy2['sequences'] = df_copy2.apply(lambda row: text_to_sequence(row['preprocessed_text'], word_to_index), axis = 1)

In [45]:
df_copy2

,id,sentiment,review,preprocessed_text,sequences
0,5814_8,1,With all this stuff going down at the moment w...,"[with, all, thi, stuff, go, down, at, the, mom...","[19, 31, 11, 584, 88, 196, 32, 2, 275, 19, 714..."
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi...","[the, classic, war, of, the, world, by, timoth...","[2, 353, 308, 5, 2, 191, 35, 3329, 5927, 7, 4,..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...,"[the, film, start, with, a, manag, nichola, be...","[2, 16, 183, 19, 4, 486, 3947, 2468, 138, 1899..."
3,3630_4,0,It must be assumed that those who praised this...,"[it, must, be, assum, that, those, who, prais,...","[8, 233, 24, 1360, 12, 162, 37, 1788, 11, 16, ..."
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...,"[superbl, trashi, and, wondrous, unpretenti, 8...","[3181, 3717, 3, 8665, 750, 13, 1199, 2, 1678, ..."
...,...,...,...,...,...
24995,3453_3,0,It seems like more consideration has gone into...,"[it, seem, like, more, consider, ha, gone, int...","[8, 114, 34, 54, 2096, 46, 868, 92, 2, 923, 34..."
24996,5064_1,0,I don't believe they made this film. Completel...,"[i, don, t, believ, they, made, thi, film, com...","[10, 99, 21, 193, 33, 104, 11, 16, 250, 1715, ..."
24997,10905_3,0,"Guy is a loser. Can't get girls, needs to buil...","[guy, is, a, loser, can, t, get, girl, need, t...","[172, 7, 4, 2138, 53, 21, 58, 186, 225, 6, 733..."
24998,10194_3,0,This 30 minute documentary Buñuel made in the ...,"[thi, 30, minut, documentari, buñuel, made, in...","[11, 929, 201, 625, 104, 9, 2, 461, 2166, 13, ..."


## Готовим данные к обучению 2

In [46]:
train, test = train_test_split(df_copy2, test_size=0.2)

In [47]:
X_train_seq = train['sequences']
y_train = train['sentiment']

In [48]:
X_train_seq

13940    [588, 6, 2, 5306, 7, 2, 1668, 652, 6, 2, 1503,...
18090    [11, 701, 14, 7, 70, 283, 18, 2, 1187, 286, 8,...
3468     [2, 8768, 449, 1580, 568, 46, 29, 101, 334, 3,...
12268    [110, 30, 37, 46, 122, 3324, 5517, 13, 2, 1307...
19363    [202, 7, 447, 30, 5, 117, 527, 129, 1433, 167,...
                               ...                        
5948     [11, 14, 7, 2484, 18, 3890, 2, 844, 192, 49, 1...
5182     [635, 1248, 1503, 5, 429, 11, 16, 223, 1140, 2...
3337     [93, 133, 6, 58, 66, 216, 230, 147, 45, 5, 2, ...
10850    [30, 5, 66, 1485, 16, 93, 239, 8, 56, 10, 15, ...
16497    [263, 2, 189, 10, 239, 11, 16, 56, 8, 448, 45,...
Name: sequences, Length: 20000, dtype: object

In [49]:
y_train

13940    1
18090    1
3468     1
12268    1
19363    1
        ..
5948     1
5182     1
3337     0
10850    1
16497    1
Name: sentiment, Length: 20000, dtype: int64

In [50]:
X_test_seq = test['sequences']
y_test = test['sentiment']

## Создаем BoW 2

In [51]:
X_train = vectorize_sequences(X_train_seq, max_words)
X_test = vectorize_sequences(X_test_seq, max_words)

In [52]:
X_train[0][:100]

array([ 0.,  0., 14.,  7.,  7.,  8.,  8.,  3.,  2.,  2.,  0.,  2.,  0.,
        1.,  0.,  0.,  2.,  1.,  2.,  2.,  1.,  0.,  0.,  0.,  1.,  1.,
        1.,  1.,  2.,  1.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,
        0.,  1.,  0.,  2.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  3.,  1.,
        1.,  0.,  1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        2.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

In [53]:
len(X_train[0])

10000

## Обучение модели с предобработкой

In [54]:
classifier.fit(X_train, y_train)

In [55]:
y_pred = classifier.predict(X_test)

## Метрики с предобработкой

In [56]:
from sklearn.metrics import classification_report

print(classification_report(y_pred, y_test))

              precision    recall  f1-score   support

           0       0.84      0.85      0.85      2479
           1       0.85      0.84      0.85      2521

    accuracy                           0.85      5000
   macro avg       0.85      0.85      0.85      5000
weighted avg       0.85      0.85      0.85      5000



In [57]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

accuracy = accuracy_score(y_test, y_pred)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Balanced_accuracy:", balanced_accuracy)

Accuracy: 0.8458
Balanced_accuracy: 0.845803581314292
